# NEPOOL IDR Drop

Implements class to batch tasks used for IDR drops.

In [1]:
import os
basepath = '/home/jupyter-engiela/la-tools-test/IDR_Drop/'
os.chdir(basepath)
print('working in directory {}.'.format(basepath))

from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
import selenium.webdriver as webdriver
from pandas.io.json import json_normalize
import datetime as dt
import pandas as pd
import numpy as np
import IDRdrop
import EPOwebscrape
#import emailscrape
import json
import math
import time

working in directory /home/jupyter-engiela/la-tools-test/IDR_Drop/.


In [2]:
def bodies_json(bodies):

    test = pd.DataFrame.from_dict(bodies, orient = 'index')

    if type(test.date[0]) == str:
            test.date = pd.to_datetime(test.date)

    last_days = max(test.date) - dt.timedelta(7) 

    sub = test[test.date > last_days]
        
    accts_success = [len(accts) > 0 for accts in sub.accts]
    accts_fail = [not val for val in accts_success]
        
    good = sub[accts_success].reset_index(drop = True)

    email_error = []

    if len(accts_fail) > 0:
        bad = sub[accts_fail].reset_index()
        mail_error = 'EMAIL_SCRAPE_ERROR.csv'

        bad.to_csv(mail_error, header = True, index = False)

    return(good)


#output_dict, filename = emailscrape.get_emails()
with open('email_bodies_05_30_2019.json', 'r') as f:
    bodies = json.load(f)

    bodies = json.loads(bodies)


good = bodies_json(bodies)

print('converted .json to dataframe')
#print(good.head())
#print('...')
#print(good.tail())

converted .json to dataframe


## Download files from EPO portal

In [ ]:
results = []
beg = time.time()

for row in range(0, len(good.accts)):
    try:
        EPOwebscrape.idr_download(row, good)
        results.append(True)
        print('done with {} of {}.'.format(row+1, len(good.accts)))
        print('---------------------------')
    except:
        print('error, accts {}.'.format(good.accts[row]))
        results.append(False)
        print('done with {} of {}.'.format(row+1, len(good.accts)))
        print('---------------------------')

end = time.time()
elapse = round(end - beg, 2)
print('downloaded {} files in {} seconds.'.format(len(good.accts), elapse))
good['success'] = results

user:  fbank-engie
password:  a362b
logging on...
set to last  400  days.
logging on...
looking for accts ['51479802043_956841004', '51386152045_404471006', '51711052035_422371006'].
found items in portal.
trying search & download...


/home/jupyter-engiela/la-tools-test/IDR_Drop/EPOwebscrape.py:164: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 164 of the file /home/jupyter-engiela/la-tools-test/IDR_Drop/EPOwebscrape.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(browser.page_source)


disabling demand...
selecting hourly interval...
submitting...
error, accts ['51479802043_956841004', '51386152045_404471006', '51711052035_422371006'].
done with 1 of 56.
---------------------------
user:  400a-engie
password:  733gg
logging on...
error, accts ['51414072009_412481000'].
done with 2 of 56.
---------------------------
user:  mar-nengie
password:  gwg153
logging on...
error, accts ['27163830014', '16107510014', '28837550020'].
done with 3 of 56.
---------------------------
user:  mar1-ndirect
password:  94arbb
logging on...
set to last  400  days.
logging on...
looking for accts ['28837550020'].
found items in portal.
trying search & download...
disabling demand...
selecting hourly interval...
submitting...
error, accts ['28837550020'].
done with 4 of 56.
---------------------------
user:  cmj1-nengie
password:  55gmgt
logging on...
set to last  400  days.
logging on...
looking for accts ['26281241005', '26281231006', '26281221007', '26281131008', '26281051008', '2628090

In [20]:
print('success with {} of {} accounts.'.format(np.sum(good.success), len(good.success)))
good.to_csv('webscrape_bodies_053019.csv')
good

success with 41 of 64 accounts.


,accts,date,name,pw,user,success
0,"[800030802, 800194201]",2019-05-30 11:44:39,NASHUA YMCA,7m5f7f,nhengi-nash3n,False
1,"[800537201, 800581301]",2019-05-30 11:42:43,PORTWALK HAMPTON INN,2n5a2a,nhengi-hamp8n,False
2,[800166101],2019-05-30 11:38:45,OATS REALTY,zu43z8,nhengi-oats6n,False
3,[800582101],2019-05-29 09:04:42,"BRITTON LUMBER COMPANY,",4s4e7s,nhengi-brit8n,False
4,"[800497301, 800497304, 800497305]",2019-05-29 09:00:45,TEXTILES COATED,7u4n4f,nhengi-text9n,False
5,[800530803],2019-05-28 09:57:19,VECTRONIX INC,uu36u5,nhengi-xinc5n,False
6,[800458501],2019-05-24 11:14:06,SEACOAST HARLEY DAVIDSON,zu43z8,nhengi-seac5n,False
7,[800084001],2019-05-24 11:13:30,WHITE MOUNTAIN LUMBER,zm53z8,nhengi-whit8n,False
8,"[51479802043_956841004, 51386152045_404471006,...",2019-05-30 13:58:02,Bank of America,a362b,fbank-engie,True
9,[51414072009_412481000],2019-05-30 11:44:59,400 Atlantic Title LLC,733gg,400a-engie,False


## Show downloaded files from EPO portal

Here *filepath* is a directory containing downloaded EPO files. Code will print 20 most recent files.

In [3]:
#filepath = os.getcwd()
readpath = os.path.join(basepath, 'Downloads')

myfiles = IDRdrop.show_dir(readpath, 65)
print(myfiles)

files found in dir:  /home/jupyter-engiela/la-tools-test/IDR_Drop/Downloads
                                                files                time
0   eversource_9eb3d10d-9099-4a77-af74-feb5429fe79... 2019-09-29 18:42:18
1   eversource_933e0ade-a40b-425d-9d0e-a2e5f650909... 2019-09-29 18:42:09
2   eversource_32abcfbb-76a4-4834-8c32-5a940ebfed4... 2019-09-29 18:41:48
3   eversource_d827c7d8-1dac-4831-a463-e1ff7ad274f... 2019-09-29 18:41:14
4   eversource_a50d870c-9fbe-42c8-94c3-c1e623ce17b... 2019-09-29 18:39:05
5   ngrid_59f04d6a-1afa-4a5f-a258-41628839725f_1a9... 2019-09-02 17:14:47
6   ngrid_80bd3467-a103-4ef8-aac4-31e5fa0327a2_325... 2019-09-02 17:13:48
7   ngrid_52a47b1f-885a-49bf-90fd-20d7f6e376ab_a24... 2019-09-02 17:12:46
8   ngrid_3fed1274-e168-4141-b91d-428428d22462_3be... 2019-09-02 17:11:45
9   ngrid_1c6a1a08-8265-49da-8726-284fc824e95c_aac... 2019-09-02 17:10:36
10  ngrid_685bfe72-bdc5-4100-a496-d0d4f475db84_23a... 2019-09-02 17:09:36
11  ngrid_065a3849-d4f9-45cb-b9e6-4b

Choose files to split into Raw IDR files.

In [5]:
index = [0]

splitfiles = list(myfiles.files[:56])
print('files to split: ')
print(splitfiles)

files to split: 
['eversource_9eb3d10d-9099-4a77-af74-feb5429fe793_851c1690_hourlycsv.csv', 'eversource_933e0ade-a40b-425d-9d0e-a2e5f6509093_605ba553_hourlycsv.csv', 'eversource_32abcfbb-76a4-4834-8c32-5a940ebfed46_901b82b7_hourlycsv.csv', 'eversource_d827c7d8-1dac-4831-a463-e1ff7ad274fc_898c92ee_hourlycsv.csv', 'eversource_a50d870c-9fbe-42c8-94c3-c1e623ce17b2_fb76e404_hourlycsv.csv', 'ngrid_59f04d6a-1afa-4a5f-a258-41628839725f_1a9b37d1_hourlycsv.csv', 'ngrid_80bd3467-a103-4ef8-aac4-31e5fa0327a2_325c35a5_hourlycsv.csv', 'ngrid_52a47b1f-885a-49bf-90fd-20d7f6e376ab_a242f888_hourlycsv.csv', 'ngrid_3fed1274-e168-4141-b91d-428428d22462_3bed57ba_hourlycsv.csv', 'ngrid_1c6a1a08-8265-49da-8726-284fc824e95c_aac16414_hourlycsv.csv', 'ngrid_685bfe72-bdc5-4100-a496-d0d4f475db84_23ac0a11_hourlycsv.csv', 'ngrid_065a3849-d4f9-45cb-b9e6-4bcf17af9435_4f6c246c_hourlycsv.csv', 'ngrid_798597cc-1c13-4ca2-b013-2373bbcdad9a_72ac3e10_hourlycsv.csv', 'ngrid_feacf514-eb12-4728-95d3-6ee55583a78c_204cb917_hourlyc

## Batch process downloaded EPO files into Raw IDRs

In [6]:
readpath = os.path.join(basepath, 'Downloads')
writepath = os.path.join(basepath, 'Raw_IDR')
utility = 'BECO'
error_log = []

for file in splitfiles:
    try:
        os.chdir(readpath)
        filedf = pd.read_csv(file, sep = ",", header = 0)
    
        IDRdrop.raw_split(filedf, readpath, writepath, utility)
        print('success, file: ', file)
        
    except:
        print('error, file: ', file)
        
print('RAW split complete.')

found 1 accounts.
27126800013_BECO_IDR_RAW.csv  written.
success, file:  eversource_9eb3d10d-9099-4a77-af74-feb5429fe793_851c1690_hourlycsv.csv
found 4 accounts.
26281131008_BECO_IDR_RAW.csv  written.
26281221007_BECO_IDR_RAW.csv  written.
26281231006_BECO_IDR_RAW.csv  written.
26281241005_BECO_IDR_RAW.csv  written.
success, file:  eversource_933e0ade-a40b-425d-9d0e-a2e5f6509093_605ba553_hourlycsv.csv
found 1 accounts.
28837550020_BECO_IDR_RAW.csv  written.
success, file:  eversource_32abcfbb-76a4-4834-8c32-5a940ebfed46_901b82b7_hourlycsv.csv
found 3 accounts.
386152045_BECO_IDR_RAW.csv  written.
479802043_BECO_IDR_RAW.csv  written.
711052035_BECO_IDR_RAW.csv  written.
success, file:  eversource_d827c7d8-1dac-4831-a463-e1ff7ad274fc_898c92ee_hourlycsv.csv
found 3 accounts.
success, file:  eversource_a50d870c-9fbe-42c8-94c3-c1e623ce17b2_fb76e404_hourlycsv.csv
found 1 accounts.
7554298017_BECO_IDR_RAW.csv  written.
success, file:  ngrid_59f04d6a-1afa-4a5f-a258-41628839725f_1a9b37d1_hourly

## Show Raw IDR files based on utility

Here *rawpath* is directory containing Raw IDRs - 25 most recent will be shown.

In [7]:
rawfiles = IDRdrop.show_dir(writepath, 75)
print(rawfiles)

files found in dir:  /home/jupyter-engiela/la-tools-test/IDR_Drop/Raw_IDR
                           files                time
0   25857301037_BECO_IDR_RAW.csv 2019-09-29 18:43:54
1   25769661015_BECO_IDR_RAW.csv 2019-09-29 18:43:54
2   26036891005_BECO_IDR_RAW.csv 2019-09-29 18:43:54
3   26272021044_BECO_IDR_RAW.csv 2019-09-29 18:43:54
4   29808330012_BECO_IDR_RAW.csv 2019-09-29 18:43:54
..                           ...                 ...
63   7554298017_BECO_IDR_RAW.csv 2019-09-29 18:43:50
64  26281131008_BECO_IDR_RAW.csv 2019-09-29 18:43:50
65  28837550020_BECO_IDR_RAW.csv 2019-09-29 18:43:50
66    479802043_BECO_IDR_RAW.csv 2019-09-29 18:43:50
67            .ipynb_checkpoints 2019-09-01 21:32:21

[68 rows x 2 columns]


Choose Raw IDRs to filter into IDR files.

In [8]:
index = [36]

processfiles = list(rawfiles.files[:67])
print('files to processed: ')
print(processfiles)

files to processed: 
['25857301037_BECO_IDR_RAW.csv', '25769661015_BECO_IDR_RAW.csv', '26036891005_BECO_IDR_RAW.csv', '26272021044_BECO_IDR_RAW.csv', '29808330012_BECO_IDR_RAW.csv', '26036761000_BECO_IDR_RAW.csv', '25770251012_BECO_IDR_RAW.csv', '29808320013_BECO_IDR_RAW.csv', '25835511020_BECO_IDR_RAW.csv', '29001880011_BECO_IDR_RAW.csv', '26572411044_BECO_IDR_RAW.csv', '26572481021_BECO_IDR_RAW.csv', '26558301011_BECO_IDR_RAW.csv', '938284031_BECO_IDR_RAW.csv', '26112151001_BECO_IDR_RAW.csv', '25658701005_BECO_IDR_RAW.csv', '29001890010_BECO_IDR_RAW.csv', '26332291033_BECO_IDR_RAW.csv', '26572421035_BECO_IDR_RAW.csv', '26312791002_BECO_IDR_RAW.csv', '29897280011_BECO_IDR_RAW.csv', '26863510017_BECO_IDR_RAW.csv', '29623140018_BECO_IDR_RAW.csv', '28307660010_BECO_IDR_RAW.csv', '25667931015_BECO_IDR_RAW.csv', '26571511034_BECO_IDR_RAW.csv', '25745431046_BECO_IDR_RAW.csv', '26572291008_BECO_IDR_RAW.csv', '288552094_BECO_IDR_RAW.csv', '399832070_BECO_IDR_RAW.csv', '285642047_BECO_IDR_RAW.

## Batch filter Raw IDR into IDR files to be dropped

In [9]:
readpath = os.path.join(basepath, 'Raw_IDR')
writepath = os.path.join(basepath, 'IDR')
error_log = []

for dropfile in processfiles:
    try:
        IDRdrop.data_drop(dropfile, readpath, writepath)
        print('success, file: ', dropfile)
    
    except:
        error_log.append(dropfile)
        print("error, file: ", dropfile)

success, file:  25857301037_BECO_IDR_RAW.csv
success, file:  25769661015_BECO_IDR_RAW.csv
success, file:  26036891005_BECO_IDR_RAW.csv
success, file:  26272021044_BECO_IDR_RAW.csv
success, file:  29808330012_BECO_IDR_RAW.csv
success, file:  26036761000_BECO_IDR_RAW.csv
success, file:  25770251012_BECO_IDR_RAW.csv
success, file:  29808320013_BECO_IDR_RAW.csv
error, file:  25835511020_BECO_IDR_RAW.csv
error, file:  29001880011_BECO_IDR_RAW.csv
success, file:  26572411044_BECO_IDR_RAW.csv
success, file:  26572481021_BECO_IDR_RAW.csv
success, file:  26558301011_BECO_IDR_RAW.csv
success, file:  938284031_BECO_IDR_RAW.csv
success, file:  26112151001_BECO_IDR_RAW.csv
success, file:  25658701005_BECO_IDR_RAW.csv
success, file:  29001890010_BECO_IDR_RAW.csv
success, file:  26332291033_BECO_IDR_RAW.csv
success, file:  26572421035_BECO_IDR_RAW.csv
success, file:  26312791002_BECO_IDR_RAW.csv
success, file:  29897280011_BECO_IDR_RAW.csv
success, file:  26863510017_BECO_IDR_RAW.csv
success, file:  

In [10]:
error_log

['25835511020_BECO_IDR_RAW.csv',
 '29001880011_BECO_IDR_RAW.csv',
 '28307660010_BECO_IDR_RAW.csv',
 '25745431046_BECO_IDR_RAW.csv',
 '288552094_BECO_IDR_RAW.csv',
 '25651191022_BECO_IDR_RAW.csv',
 '26281241005_BECO_IDR_RAW.csv',
 '26281231006_BECO_IDR_RAW.csv',
 '26281221007_BECO_IDR_RAW.csv',
 '26281131008_BECO_IDR_RAW.csv']